# Milvus Implemented

In [ ]:
import os
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import pymilvus
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility
print("All imports were successful!")


In [ ]:
#resnet18 used
model=models.resnet18(pretrained=True)
model = nn.Sequential(*list(model.children())[:1])
model.eval()

In [ ]:
def preprocess_image(image_path):
    image=Image.open(image_path).convert('RGB')
    preprocess = transforms.Compose([
        transforms.Resize(224,224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return preprocess(image).unsqueeze(0)

def create_image_embedding(image_path):
    try:
        input_tensor=preprocess_image(image_path)
        with torch.no_grad():
            embeddings = model(input_tensor)
        return embeddings.squeeze().numpy()
    except Exception as e:
        print("Error:")

In [ ]:
connections.connect("default", host="localhost", port="19530")

In [ ]:
fields = [
    FieldSchema(name="pk", dtype=DataType.INT64, isprimary=True, auto_id=True),
    FieldSchema(name="image_id", dtype=DataType.VARCHAR, max_length=65535),
    FieldSchema(name="image_emd", dtype=DataType.FLOAT_VECTOR, dim=512)
]

schema = CollectionSchema(fields, "Implementing Milvus for Similar Search")
image = Collection("image", schema)


In [ ]:
#creating dataset

image_dataset_folder = "/Users/rishabhsaggar/Downloads/Photos" #add directory
image_paths = [os.path.join(image_dataset_folder, fname) for fname in os.listdir(image_dataset_folder) if fname.endswith(('.jpg', '.jpeg', '.png'))]

for idx in range(len(dataset)):


    # Insert vector into Milvus
    entities = [{"image_vector": vector.tolist()}]
    collection.insert(entities)
    collection.flush()
    collection.load()


In [ ]:
#searching
target_image="/add/your/photo-path"
results=collection.search(
    data=[target_image],
    anns_field="image_emd",
    param=search_params,
    limit=4,
    expr=None,

    output_fields=['image_id']
    consistency_level="Strong"
    )
